## 1. Configuration

In [ ]:
WORKSPACE_ID = "id"

BRONZE_LAKEHOUSE_ID = "id"  # lh_bronze ID
SILVER_LAKEHOUSE_ID = "id"  # lh_silver ID
GOLD_LAKEHOUSE_ID = "id"      # lh_gold ID

# Construct ABFSS paths
ONELAKE_BASE = f"abfss://{WORKSPACE_ID}@onelake.dfs.fabric.microsoft.com"

BRONZE_PATH = f"{ONELAKE_BASE}/{BRONZE_LAKEHOUSE_ID}"
SILVER_PATH = f"{ONELAKE_BASE}/{SILVER_LAKEHOUSE_ID}"
GOLD_PATH = f"{ONELAKE_BASE}/{GOLD_LAKEHOUSE_ID}"

print("📍 Lakehouse Paths:")
print(f"  Bronze: {BRONZE_PATH}")
print(f"  Silver: {SILVER_PATH}")
print(f"  Gold:   {GOLD_PATH}")

## 2. Create Bronze Layer Tables

In [ ]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    TimestampType,
    DoubleType,
    IntegerType,
    BooleanType,
    ArrayType,
    LongType,
)

### 2.1 Bronze: raw_steam_store

In [ ]:
bronze_store_schema = StructType([
    # ===== Metadata fields =====
    StructField("record_id", StringType(), False),
    StructField("batch_id", StringType(), False),
    StructField("source", StringType(), False),
    StructField("source_endpoint", StringType(), True),
    StructField("ingested_at", TimestampType(), False),
    StructField("api_response_time_ms", DoubleType(), True),
    StructField("status", StringType(), False),
    StructField("error_message", StringType(), True),
    StructField("environment", StringType(), False),
    StructField("pipeline_version", StringType(), False),

    # ===== Business key =====
    StructField("app_id", IntegerType(), False),

    # ===== Raw game data =====
    StructField("name", StringType(), True),
    StructField("type", StringType(), True),
    StructField("is_free", BooleanType(), True),
    StructField("short_description", StringType(), True),
    StructField("developers", ArrayType(StringType()), True),
    StructField("publishers", ArrayType(StringType()), True),

    # Price
    StructField("price_currency", StringType(), True),
    StructField("price_initial", IntegerType(), True),
    StructField("price_final", IntegerType(), True),
    StructField("price_discount_percent", IntegerType(), True),

    # Platforms
    StructField("platforms_windows", BooleanType(), True),
    StructField("platforms_mac", BooleanType(), True),
    StructField("platforms_linux", BooleanType(), True),

    # Metadata
    StructField("metacritic_score", IntegerType(), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("genres", ArrayType(StringType()), True),
    StructField("release_date", StringType(), True),
    StructField("coming_soon", BooleanType(), True),

    # Full raw response for reprocessing
    StructField("raw_json", StringType(), True),

    # ===== Partition column =====
    StructField("ingestion_date", StringType(), False),
])

# Create empty table with schema
bronze_store_df = spark.createDataFrame([], bronze_store_schema)

bronze_store_table = f"{BRONZE_PATH}/Tables/raw_steam_store"
bronze_store_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .option("overwriteSchema", "true") \
    .save(bronze_store_table)

print(f"Created: {bronze_store_table}")

### 2.2 Bronze: raw_steam_reviews

In [ ]:
bronze_reviews_schema = StructType([
    # ===== Metadata fields =====
    StructField("record_id", StringType(), False),
    StructField("batch_id", StringType(), False),
    StructField("source", StringType(), False),
    StructField("source_endpoint", StringType(), True),
    StructField("ingested_at", TimestampType(), False),
    StructField("api_response_time_ms", DoubleType(), True),
    StructField("status", StringType(), False),
    StructField("error_message", StringType(), True),
    StructField("environment", StringType(), False),
    StructField("pipeline_version", StringType(), False),

    # ===== Business key =====
    StructField("app_id", IntegerType(), False),

    # ===== Review summary data =====
    StructField("total_reviews", IntegerType(), True),
    StructField("total_positive", IntegerType(), True),
    StructField("total_negative", IntegerType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_score_desc", StringType(), True),

    # Full raw response
    StructField("raw_json", StringType(), True),

    # ===== Partition column =====
    StructField("ingestion_date", StringType(), False),
])

bronze_reviews_df = spark.createDataFrame([], bronze_reviews_schema)

bronze_reviews_table = f"{BRONZE_PATH}/Tables/raw_steam_reviews"
bronze_reviews_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .option("overwriteSchema", "true") \
    .save(bronze_reviews_table)

print(f"Created: {bronze_reviews_table}")

### 2.3 Bronze: raw_steam_player_stats

In [ ]:
bronze_players_schema = StructType([
    # ===== Metadata fields =====
    StructField("record_id", StringType(), False),
    StructField("batch_id", StringType(), False),
    StructField("source", StringType(), False),
    StructField("source_endpoint", StringType(), True),
    StructField("ingested_at", TimestampType(), False),
    StructField("api_response_time_ms", DoubleType(), True),
    StructField("status", StringType(), False),
    StructField("error_message", StringType(), True),
    StructField("environment", StringType(), False),
    StructField("pipeline_version", StringType(), False),

    # ===== Business key =====
    StructField("app_id", IntegerType(), False),

    # ===== Player data =====
    StructField("player_count", IntegerType(), True),

    # Full raw response
    StructField("raw_json", StringType(), True),

    # ===== Partition column =====
    StructField("ingestion_date", StringType(), False),
])

bronze_players_df = spark.createDataFrame([], bronze_players_schema)

bronze_players_table = f"{BRONZE_PATH}/Tables/raw_steam_player_stats"
bronze_players_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .option("overwriteSchema", "true") \
    .save(bronze_players_table)

print(f"Created: {bronze_players_table}")

## 3. Create Silver Layer Tables (SCD Type 2)

### 3.1 Silver: dim_games (SCD Type 2)

In [ ]:
silver_games_schema = StructType([
    # ===== Surrogate key (for SCD2) =====
    StructField("game_sk", LongType(), False),

    # ===== Natural key =====
    StructField("app_id", IntegerType(), False),

    # ===== Tracked attributes =====
    StructField("name", StringType(), True),
    StructField("type", StringType(), True),
    StructField("is_free", BooleanType(), True),
    StructField("short_description", StringType(), True),
    StructField("developers", ArrayType(StringType()), True),
    StructField("publishers", ArrayType(StringType()), True),

    # Price (main SCD2 driver - we track price changes)
    StructField("price_currency", StringType(), True),
    StructField("price_initial_cents", IntegerType(), True),
    StructField("price_final_cents", IntegerType(), True),
    StructField("price_discount_percent", IntegerType(), True),

    # Platforms
    StructField("platforms_windows", BooleanType(), True),
    StructField("platforms_mac", BooleanType(), True),
    StructField("platforms_linux", BooleanType(), True),

    # Classification
    StructField("metacritic_score", IntegerType(), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("genres", ArrayType(StringType()), True),

    # Release info
    StructField("release_date", StringType(), True),
    StructField("coming_soon", BooleanType(), True),

    # ===== SCD Type 2 fields =====
    StructField("valid_from", TimestampType(), False),
    StructField("valid_to", TimestampType(), False),  # 9999-12-31 for current
    StructField("is_current", BooleanType(), False),

    # ===== Audit fields =====
    StructField("_created_at", TimestampType(), False),
    StructField("_updated_at", TimestampType(), False),
    StructField("_source_batch_id", StringType(), True),
])

silver_games_df = spark.createDataFrame([], silver_games_schema)

silver_games_table = f"{SILVER_PATH}/Tables/dim_games"
silver_games_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("is_current") \
    .option("overwriteSchema", "true") \
    .save(silver_games_table)

print(f"Created: {silver_games_table}")

### 3.2 Silver: dim_game_reviews (SCD Type 2)

In [ ]:
silver_reviews_schema = StructType([
    # ===== Surrogate key =====
    StructField("review_sk", LongType(), False),

    # ===== Natural key =====
    StructField("app_id", IntegerType(), False),

    # ===== Tracked attributes =====
    StructField("total_reviews", IntegerType(), True),
    StructField("total_positive", IntegerType(), True),
    StructField("total_negative", IntegerType(), True),
    StructField("positive_ratio", DoubleType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_score_desc", StringType(), True),

    # ===== SCD Type 2 fields =====
    StructField("valid_from", TimestampType(), False),
    StructField("valid_to", TimestampType(), False),
    StructField("is_current", BooleanType(), False),

    # ===== Audit fields =====
    StructField("_created_at", TimestampType(), False),
    StructField("_updated_at", TimestampType(), False),
    StructField("_source_batch_id", StringType(), True),
])

silver_reviews_df = spark.createDataFrame([], silver_reviews_schema)

silver_reviews_table = f"{SILVER_PATH}/Tables/dim_game_reviews"
silver_reviews_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("is_current") \
    .option("overwriteSchema", "true") \
    .save(silver_reviews_table)

print(f"Created: {silver_reviews_table}")

### 3.3 Silver: fact_player_counts (Snapshot Fact)

In [ ]:
silver_players_schema = StructType([
    # ===== Keys =====
    StructField("app_id", IntegerType(), False),
    StructField("snapshot_timestamp", TimestampType(), False),

    # ===== Measures =====
    StructField("player_count", IntegerType(), True),

    # ===== Audit fields =====
    StructField("_created_at", TimestampType(), False),
    StructField("_source_batch_id", StringType(), True),

    # ===== Partition column =====
    StructField("snapshot_date", StringType(), False),
])

silver_players_df = spark.createDataFrame([], silver_players_schema)

silver_players_table = f"{SILVER_PATH}/Tables/fact_player_counts"
silver_players_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("snapshot_date") \
    .option("overwriteSchema", "true") \
    .save(silver_players_table)

print(f"Created: {silver_players_table}")

## 4. Create Gold Layer Tables

### 4.1 Gold: agg_game_metrics

In [ ]:
schema_game_metrics = StructType([
    StructField("app_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("type", StringType(), True),
    StructField("is_free", BooleanType(), True),
    StructField("price_currency", StringType(), True),
    StructField("price_initial_cents", IntegerType(), True),
    StructField("price_final_cents", IntegerType(), True),
    StructField("price_discount_percent", IntegerType(), True),
    StructField("price_usd", DoubleType(), True),
    StructField("is_on_sale", BooleanType(), True),
    StructField("genres", ArrayType(StringType()), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("developers", ArrayType(StringType()), True),
    StructField("publishers", ArrayType(StringType()), True),
    StructField("platforms_windows", BooleanType(), True),
    StructField("platforms_mac", BooleanType(), True),
    StructField("platforms_linux", BooleanType(), True),
    StructField("metacritic_score", IntegerType(), True),
    StructField("total_reviews", IntegerType(), True),
    StructField("total_positive", IntegerType(), True),
    StructField("total_negative", IntegerType(), True),
    StructField("positive_ratio", DoubleType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_score_desc", StringType(), True),
    StructField("current_players", IntegerType(), True),
    StructField("player_count_updated_at", TimestampType(), True),
    StructField("popularity_score", DoubleType(), True),
    StructField("release_date", StringType(), True),
    StructField("coming_soon", BooleanType(), True),
    StructField("_refreshed_at", TimestampType(), False),
])

spark.createDataFrame([], schema_game_metrics).write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{GOLD_PATH}/Tables/agg_game_metrics")

### 4.2 Gold: agg_price_history

In [ ]:
schema_price_history = StructType([
    StructField("app_id", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("price_effective_from", TimestampType(), True),
    StructField("price_effective_to", TimestampType(), True),
    StructField("is_current", BooleanType(), True),
    StructField("price_currency", StringType(), True),
    StructField("price_initial_cents", IntegerType(), True),
    StructField("price_final_cents", IntegerType(), True),
    StructField("price_discount_percent", IntegerType(), True),
    StructField("price_initial_usd", DoubleType(), True),
    StructField("price_final_usd", DoubleType(), True),
    StructField("discount_amount_usd", DoubleType(), True),
    StructField("is_discounted", BooleanType(), True),
    StructField("discount_tier", StringType(), True),
    StructField("genres", ArrayType(StringType()), True),
    StructField("_refreshed_at", TimestampType(), False),
    StructField("effective_date", StringType(), True),
])

spark.createDataFrame([], schema_price_history).write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("is_current") \
    .save(f"{GOLD_PATH}/Tables/agg_price_history")

### 4.3 Gold: agg_genre_summary

In [ ]:
schema_genre_summary = StructType([
    StructField("genre", StringType(), False),
    StructField("game_count", LongType(), True),
    StructField("free_games_count", LongType(), True),
    StructField("avg_price_usd", DoubleType(), True),
    StructField("min_price_usd", DoubleType(), True),
    StructField("max_price_usd", DoubleType(), True),
    StructField("avg_discount_percent", DoubleType(), True),
    StructField("games_on_sale_count", LongType(), True),
    StructField("total_reviews", LongType(), True),
    StructField("avg_positive_ratio", DoubleType(), True),
    StructField("avg_review_score", DoubleType(), True),
    StructField("avg_metacritic_score", DoubleType(), True),
    StructField("games_with_metacritic", LongType(), True),
    StructField("total_current_players", LongType(), True),
    StructField("avg_players_per_game", DoubleType(), True),
    StructField("max_players_single_game", IntegerType(), True),
    StructField("percent_free", DoubleType(), True),
    StructField("percent_on_sale", DoubleType(), True),
    StructField("_refreshed_at", TimestampType(), False),
])

spark.createDataFrame([], schema_genre_summary).write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{GOLD_PATH}/Tables/agg_genre_summary")